### Task #1: Get info box (store in Python dictionary)

#### Import necessary libraries

In [4]:
from bs4 import BeautifulSoup as bs
import requests

In [12]:
r = requests.get('https://en.wikipedia.org/wiki/Inspector_Gadget_(film)')

# Convert to a beautiful soup object
soup = bs(r.content)

# Getting the HTML code
contents = soup.prettify()

In [13]:
# Finding infobox class to get table info from wiki
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all('tr')
  

In [17]:
def get_content_value(row_data):
  if row_data.find('li'):
    return [li.get_text(" ", strip=True).replace('\xa0', ' ') for li in row_data('li')]
  else:
    return row_data.get_text(" ", strip=True).replace('\xa0', ' ')

# Creating a set with data info
movie_info = {}
for index, row in enumerate(info_rows):
  if index == 0:
    movie_info['title'] = row.find('th').get_text(" ", strip=True)
  elif index == 1:
    continue
  else:
    content_key = row.find('th').get_text(" ", strip=True)
    content_value = get_content_value(row.find('td'))
    movie_info[content_key] = content_value

movie_info

{'title': 'Inspector Gadget',
 'Directed by': 'David Kellogg',
 'Screenplay by': ['Kerry Ehrin', 'Zak Penn', 'Audrey Wells (uncredited)'],
 'Story by': ['Kerry Ehrin', 'Dana Olsen'],
 'Based on': 'Inspector Gadget by Bruno Bianchi Jean Chalopin Andy Heyward',
 'Produced by': ['Roger Birnbaum', 'Andy Heyward', 'Jordan Kerner'],
 'Starring': ['Matthew Broderick',
  'Rupert Everett',
  'Joely Fisher',
  'Michelle Trachtenberg',
  'Mike Hagerty',
  'Andy Dick',
  'Cheri Oteri',
  'Dabney Coleman'],
 'Cinematography': 'Adam Greenberg',
 'Edited by': ['Alan Cody', 'Thom Noble'],
 'Music by': 'John Debney',
 'Production companies': ['Walt Disney Pictures [1]',
  'Caravan Pictures',
  'DIC Entertainment',
  'Avnet/Kerner Productions',
  'Roger Birnbaum Productions'],
 'Distributed by': 'Buena Vista Pictures Distribution [1]',
 'Release date': ['July 23, 1999 ( 1999-07-23 )'],
 'Running time': '78 minutes [2]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$75–90 million [3] 

### Task #2: Get info box for all movies

In [18]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films')

# Convert to a beautiful soup object
soup = bs(r.content)

# Getting the HTML code
contents = soup.prettify()

In [31]:
# Getting movie link
movies = soup.select('.wikitable.sortable i')
movies[0:10]

[<i><a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a></i>,
 <i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>,
 <i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>,
 <i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>,
 <i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>,
 <i><a href="/wiki/Dumbo" title="Dumbo">Dumbo</a></i>,
 <i><a href="/wiki/Bambi" title="Bambi">Bambi</a></i>,
 <i><a href="/wiki/Saludos_Amigos" title="Saludos Amigos">Saludos Amigos</a></i>,
 <i><a href="/wiki/Victory_Through_Air_Power_(film)" title="Victory Through Air Power (film)">Victory Through Air Power</a></i>,
 <i><a href="/wiki/The_Three_Caballeros" title=

In [32]:
def get_info_box(url):
  r = requests.get(url)

  # Convert to a beautiful soup object and getting rows
  soup = bs(r.content)
  info_box = soup.find(class_="infobox vevent")
  info_rows = info_box.find_all('tr')

  # Creating a set with data info
  movie_info = {}
  for index, row in enumerate(info_rows):
    if index == 0:
      movie_info['title'] = row.find('th').get_text(" ", strip=True)
    elif index == 1:
      continue
    else:
      content_key = row.find('th').get_text(" ", strip=True)
      content_value = get_content_value(row.find('td'))
      movie_info[content_key] = content_value

  return movie_info

In [33]:
base_path = 'https://en.wikipedia.org/'
movie_info_list = []
for index, movie in enumerate(movies):
  if index == 10:
    break
  try:
    relative_path = movie.a['href']
    full_path = base_path + relative_path
    title = movie.a['title']

    movie_info_list.append(get_info_box(full_path))

  except Exception as e:
    print(f"[err] {e} >> {movie.get_text()}")

In [37]:
len(movie_info_list)

10

Save/Reload Movie data

In [41]:
import json

def save_data(title, data):
  with open(title, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [42]:
def load_data(title):
  with open(title, encoding='utf-8') as f:
    return json.load(f)

In [46]:
save_data('disney_data.json', movie_info_list)